<a href="https://colab.research.google.com/github/gmrammohan15/EVA4/blob/master/S11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import sys
sys.path.append('/content/gdrive/My Drive/S10-Assignment/')
!ls /content/gdrive/My\ Drive/S10-Assignment

data		    grad_cam.py       Models			 utils.py
dataloader.py	    __init__.py       __pycache__
data_transforms.py  lr_finder.py      README.md
eval.py		    misclassified.py  Transforms_Albumentations


In [3]:
%cd /content/gdrive/My Drive/S10-Assignment/

/content/gdrive/My Drive/S10-Assignment


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
!pip install albumentations

     |████████████████████████████████| 634kB 8.8MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=f0dfefb006c58972546ac11e68734683a905ea3ef4bce7dba0a091f08b5be83a
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [5]:
from Transforms_Albumentations import transform

import importlib
importlib.reload(transform)

#from transform  import Data
data = transform.Data()

trainset = data.getTrainDataSet(train=True)
testset =  data.getTrainDataSet(train=False)
trainloader = data.getDataLoader(trainset, 128)
show_trainloader = data.getDataLoader(trainset, 4)
testloader = data.getDataLoader(testset, 128)
show_testloader = data.getDataLoader(testset, 4)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [0]:
import torch.nn as nn
import torch.nn.functional as F

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1,
                     padding=1)
    
def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        if self.downsample is not None:
            residual = self.downsample(x)

        out = self.conv1(residual)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += residual
        out = self.relu(out)

        return out

class CustomResNet(nn.Module):
    def __init__(self):
        super(CustomResNet, self).__init__()

        block = BasicBlock
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)


        # self.convy = nn.Conv2d(64, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)
        # self.bny = nn.BatchNorm2d(self.inplanes)
        # self.maxpooly = nn.MaxPool2d(2,2)
        # self.reluy = nn.ReLU(inplace=True)

        self.convx = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)
        self.bnx = nn.BatchNorm2d(256)
        self.maxpoolx = nn.MaxPool2d(2,2)
        self.relux = nn.ReLU(inplace=True)

        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        self.layer1 = self._make_layer(block, 128, 1)
        self.layer2 = self._make_layer(block, 512, 1, inplanes=256)

        self.maxpool4 = nn.MaxPool2d(kernel_size=4, stride=1, padding=0)

        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(512 , 10)


    def _make_layer(self, block, planes, blocks, stride=1, inplanes=64):
        downsample = None
        self.inplanes = inplanes
        downsample = nn.Sequential(
            nn.Conv2d(self.inplanes, planes ,
                      kernel_size=3, padding=1, stride=stride, bias=False),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(planes),
            nn.ReLU()
        )

        layers = []

        
        self.inplanes = planes

        layers.append(block(self.inplanes, planes, stride, downsample))
        
        # for i in range(1, blocks):
        #     layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):

      x = self.conv1(x)
      x = self.bn1(x)
      x = self.relu(x)
      

      # self.inplanes = 128

      # x = self.convy(x)
      # x = self.maxpooly(x)
      # x = self.bny(x)
      # x = self.reluy(x)

      x = self.layer1(x)

      self.inplanes = 256

      x = self.convx(x)
      x = self.maxpoolx(x)
      x = self.bnx(x)
      x = self.relux(x)

      self.inplanes = 256
      x = self.layer2(x)

      # x = self.maxpool4(x)

      # x = self.fc(x)

      # return F.log_softmax(x, dim=-1)


In [68]:

import torch
!pip install torchsummary
from torchsummary import summary

model = CustomResNet()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = CustomResNet().to(device)
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]          73,728
         MaxPool2d-5          [-1, 128, 16, 16]               0
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
       BatchNorm2d-9          [-1, 128, 16, 16]             256
             ReLU-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,584
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1, 128, 16, 16]               0
       BasicBlock-14          [-1, 128,